In [147]:
import numpy as np
import pandas as pd

## Question 12

In [148]:
def load_data(file):
    data = pd.read_csv(file, header=None, sep=' ').to_numpy()
    return data[:, :-1], data[:, -1]

In [149]:
class WeightedStump:
    def __init__(self):
        self.s = -1
        self.d = -1
        self.theta = -float('inf')
    
    
    def fit(self, X, y, u):
        N, D = X.shape
        Etrain = float('inf')
        eps = -1
        
        for d in range(D):
#             inds = np.argsort(X[:,d])
#             X, y = X[inds], y.reshape(N, 1)[inds].reshape(N,)
            
            feats = np.sort(np.unique(X[:,d]))
            thetas = []
            for a, b in zip(feats[:-1], feats[1:]):
                thetas.append((b+a)/2)
            thetas = [feats[0]-1, *thetas]
            
#             print(thetas)
            
            for theta in thetas:
                for s in {-1, 1}:
                    yhat = s * np.sign(X[:,d]-theta)
                    E = np.average(u * (yhat != y))
                    if E < Etrain:
#                         print('(*)', end=' ')
                        Etrain = E
                        self.s = s
                        self.d = d
                        self.theta = theta
#                     print(E)
        
        return Etrain
    
    
    def predict(self, X):
        return self.s * np.sign(X[:, self.d]-self.theta)

In [150]:
Xtrain = np.array([[1, 1], [2, 1], [3, 1], [4, 1]])
ytrain = np.array([1, -1, -1, 1])
clf = WeightedStump()
clf.fit(Xtrain, ytrain, np.array([1, 1000, 100, 100]))
clf.predict(Xtrain)

array([-1., -1., -1.,  1.])

In [167]:
Xtrain, ytrain = load_data('hw2_adaboost_train.dat')
u = np.ones(len(Xtrain)) / len(Xtrain)
gs = []
alphas = []
us = []
epss = []
for t in range(300):
    us.append(u.copy())
    clf = WeightedStump()
    Etrain = clf.fit(Xtrain, ytrain, u)
    gs.append(clf)
    
    eps_t = Etrain * len(Xtrain) / np.sum(u)
    scale = np.sqrt((1 - eps_t) / eps_t)
    alphas.append(np.log(scale))
    epss.append(eps_t)
    
    yhat = clf.predict(Xtrain)
    for i in range(len(Xtrain)):
        if yhat[i] == ytrain[i]:
            u[i] /= scale
        else:
            u[i] *= scale
            
    
    if t == 0:  # Q12
        print('Ein(g1) =', np.average(yhat != ytrain))

Ein(g1) = 0.24


In [162]:
# Predict
yhat = np.zeros(ytrain.shape)
for t in range(300):
    yhat += alphas[t] * gs[t].predict(Xtrain)
yhat = np.sign(yhat)
print('Ein(G) =', np.average(yhat != ytrain))

Ein(G) = 0.0


In [164]:
np.sum(us[1])

0.8541662601625051

In [168]:
min(epss)

0.1787280701754386

In [169]:
Xtest, ytest = load_data('hw2_adaboost_test.dat')

In [174]:
# Predict
yhat = np.zeros(ytest.shape)
for t in range(300):
    yhat += alphas[t] * gs[t].predict(Xtest)
yhat = np.sign(yhat)
print('Eout(G) =', np.average(yhat != ytest))

Eout(G) = 0.132


## Question 19

In [222]:
data = pd.read_csv('hw2_lssvm_all.dat', header=None, sep=' ')
data[data.columns[0]] = 1
data = data.to_numpy()
X, y = data[:,:-1], data[:,-1]
Xtrain, ytrain = X[:400,:], y[:400]
Xtest, ytest = X[400:,:], y[400:]

In [228]:
gammas = [32, 2, 0.125]
lambdas = [0.001, 1, 1000]

Ein = float('inf')
Eout = float('inf')

# Train
for g in gammas:
    Nt = len(Xtrain)
    K = np.zeros((Nt, Nt))
    for i in range(Nt):
        for j in range(Nt):
            K[i,j] = np.exp(-g*(np.linalg.norm(Xtrain[i] - Xtrain[j]) ** 2))
    
    for l in lambdas:
        beta = np.linalg.pinv(l*np.eye(Nt) + K) @ ytrain
        
        # Test
        yhat = np.zeros(Nt)
        for i in range(Nt):
            for j in range(Nt):
                yhat[i] += beta[j] * K[i,j]
        yhat = np.sign(yhat)
        E = np.average(yhat != ytrain)
        Ein = min(E, Ein)
        
        yhat = np.zeros(len(Xtest))
        for i in range(len(Xtest)):
            for j in range(Nt):
                yhat[i] += beta[j] * np.exp(-g*(np.linalg.norm(Xtest[i] - Xtrain[j]) ** 2))
        yhat = np.sign(yhat)
        E = np.average(yhat != ytest)
        Eout = min(E, Eout)

print(Ein)
print(Eout)

0.0
0.39
